In [20]:
import requests, json
from datetime import datetime, timedelta, timezone

BASE = "http://127.0.0.1:8765"  # change if you used a different port

def status():
    try:
        s = requests.get(f"{BASE}/status", timeout=5).json()
        print("Helper status:", s)
        if not s.get("authorized", False):
            print("⚠️  Helper is not authorized yet. Open System Settings → Privacy & Security → Calendars and enable it.")
        return s
    except Exception as e:
        print("Could not reach helper:", e)

_ = status()


Helper status: {'authorized': True, 'status_code': 3}


In [2]:
import time

# 1) choose a title and time window
TITLE = "Notebook Test Event"
start = (datetime.now().astimezone() + timedelta(minutes=5)).isoformat()
end   = (datetime.now().astimezone() + timedelta(minutes=35)).isoformat()

# 2) create the event via helper API
added = requests.post(f"{BASE}/add", json={
    "title": TITLE,
    "start_iso": start,
    "end_iso": end,
    "notes": "created from Jupyter"
}, timeout=10).json()

print("Added:", added)
EVENT_ID = added.get("id")
assert EVENT_ID, "No event id returned from helper."

# 3) verify it's present by fetching upcoming events and finding the id
def find_event_by_id(eid, days=3):
    events = requests.get(f"{BASE}/events", params={"days": days}, timeout=10).json()
    for e in events:
        if e.get("id") == eid:
            return e
    return None

# optionally wait a moment in case Calendar refresh is slow
time.sleep(1.0)
found = find_event_by_id(EVENT_ID, days=3)
print("Found:", found)

assert found is not None, "Event not found in upcoming events. Try increasing 'days' or waiting a few seconds and re-run the find."


Added: {'title': 'Notebook Test Event', 'start_iso': '2025-10-02T11:36:31.023947-04:00', 'end_iso': '2025-10-02T12:06:31.024008-04:00', 'id': '9CB16157-F8D3-456A-B211-5037BE00DE43', 'calendar': 'Home'}
Found: {'title': 'Notebook Test Event', 'start_iso': '2025-10-02T11:36:31-04:00', 'end_iso': '2025-10-02T12:06:31-04:00', 'id': '9CB16157-F8D3-456A-B211-5037BE00DE43', 'calendar': 'Home'}


In [3]:
assert EVENT_ID, "Run the add cell first to set EVENT_ID."

# 1) delete via helper
deleted = requests.post(f"{BASE}/delete", params={"event_id": EVENT_ID}, timeout=10).json()
print("Delete response:", deleted)

# 2) verify it’s gone
gone = find_event_by_id(EVENT_ID, days=3)
print("Still present?", gone is not None)
assert gone is None, "Event still present—give it a few seconds and re-run this cell."


Delete response: {'deleted': True}
Still present? False


In [4]:
import requests
from datetime import datetime, timedelta

BASE = "http://127.0.0.1:8765"

# Choose your tags here
tags = ["work", "deepwork", "priority-high"]

# Title with visible tags so they show up in /events listing
TITLE = f"Notebook Tagged Event [{ ' '.join('#' + t for t in tags) }]"

start = (datetime.now().astimezone() + timedelta(minutes=10)).isoformat()
end   = (datetime.now().astimezone() + timedelta(minutes=40)).isoformat()

# Put tags in notes in a machine-friendly way too (so later you can parse them).
notes = "tags: " + ",".join(tags) + "\nsource: jupyter"

added_tagged = requests.post(f"{BASE}/add", json={
    "title": TITLE,
    "start_iso": start,
    "end_iso": end,
    "notes": notes
}, timeout=10).json()

print("ADDED (tagged):", added_tagged)
TAGGED_EVENT_ID = added_tagged.get("id")

# Quick verify it appears in upcoming events
events_next = requests.get(f"{BASE}/events", params={"days": 3}, timeout=10).json()
found = next((e for e in events_next if e.get("id") == TAGGED_EVENT_ID), None)
print("Found newly-added tagged event:", bool(found), "\n", found)


ADDED (tagged): {'title': 'Notebook Tagged Event [#work #deepwork #priority-high]', 'start_iso': '2025-10-02T11:46:24.781581-04:00', 'end_iso': '2025-10-02T12:16:24.781636-04:00', 'id': '051963C2-36BD-499C-BC8C-B7258F9AA91A', 'calendar': 'Home'}
Found newly-added tagged event: True 
 {'title': 'Notebook Tagged Event [#work #deepwork #priority-high]', 'start_iso': '2025-10-02T11:46:24-04:00', 'end_iso': '2025-10-02T12:16:24-04:00', 'id': '051963C2-36BD-499C-BC8C-B7258F9AA91A', 'calendar': 'Home'}


In [5]:
import requests
from datetime import datetime
from pprint import pprint

BASE = "http://127.0.0.1:8765"

week_events = requests.get(f"{BASE}/events", params={"days": 7}, timeout=10).json()

# Pretty print as a simple table
print(f"Found {len(week_events)} events in the next 7 days:\n")
print(f"{'Start':<20} {'End':<20} {'Title':<60} {'ID':<25}")
print("-" * 130)
for e in week_events:
    start = datetime.fromisoformat(e["start_iso"]).strftime("%Y-%m-%d %H:%M")
    end   = datetime.fromisoformat(e["end_iso"]).strftime("%Y-%m-%d %H:%M")
    title = (e["title"] or "").replace("\n"," ")[:58]
    eid   = (e["id"] or "")[:24]
    print(f"{start:<20} {end:<20} {title:<60} {eid:<25}")

# Optional: keep IDs of the events we just added so we can delete later
# TAGGED_EVENT_ID already set if you ran the previous cell.


Found 4 events in the next 7 days:

Start                End                  Title                                                        ID                       
----------------------------------------------------------------------------------------------------------------------------------
2025-10-02 00:00     2025-10-03 00:00     Yom Kippur                                                   8F09EAB1-3910-4D16-991A- 
2025-10-02 11:46     2025-10-02 12:16     Notebook Tagged Event [#work #deepwork #priority-high]       051963C2-36BD-499C-BC8C- 
2025-10-06 00:00     2025-10-06 01:00     TA Office Hours                                              AC7A3416-9489-4CC0-B959- 
2025-10-07 16:10     2025-10-07 18:00     TA: Practicum                                                78321BA2-61E0-4B90-9BBC- 


In [6]:
import requests, collections
from datetime import datetime

BASE = "http://127.0.0.1:8765"

events = requests.get(f"{BASE}/events", params={"days": 30}, timeout=10).json()
cals = sorted({e.get("calendar","") for e in events if e.get("calendar")})
print("Calendars found from events in the next 30 days:")
for name in cals:
    print("-", name if name else "(unnamed)")


Calendars found from events in the next 30 days:
- Home
- US Holidays
- Work


In [8]:
import requests, collections
from datetime import datetime

BASE = "http://127.0.0.1:8765"

events = requests.get(f"{BASE}/events", params={"days": 7}, timeout=10).json()
by_cal = collections.defaultdict(list)
for e in events:
    by_cal[e.get("calendar","(unknown)")].append(e)

for cal_name in sorted(by_cal.keys()):
    print(f"\n=== {cal_name} ===")
    for e in sorted(by_cal[cal_name], key=lambda x: x["start_iso"]):
        s = datetime.fromisoformat(e["start_iso"]).strftime("%Y-%m-%d %H:%M")
        en = datetime.fromisoformat(e["end_iso"]).strftime("%H:%M")
        print(f"- {s} → {en} | {e['title']} (id: {e['id'][:24]})")



=== Home ===
- 2025-10-06 00:00 → 01:00 | TA Office Hours (id: AC7A3416-9489-4CC0-B959-)
- 2025-10-07 16:10 → 18:00 | TA: Practicum (id: 78321BA2-61E0-4B90-9BBC-)

=== US Holidays ===
- 2025-10-02 00:00 → 00:00 | Yom Kippur (id: 8F09EAB1-3910-4D16-991A-)

=== Work ===
- 2025-10-02 11:46 → 12:16 | Notebook Tagged Event [#work #deepwork #priority-high] (id: 051963C2-36BD-499C-BC8C-)


In [9]:
import requests
from datetime import datetime, timedelta

BASE = "http://127.0.0.1:8765"

# choose simple test times: 10:00–10:30 local time
def day_at(hour, minute, days_from_now):
    dt = datetime.now().astimezone().replace(hour=hour, minute=minute, second=0, microsecond=0)
    return (dt + timedelta(days=days_from_now))

tomorrow_start = day_at(10, 0, 1);  tomorrow_end = tomorrow_start + timedelta(minutes=30)
dayafter_start = day_at(10, 0, 2);  dayafter_end = dayafter_start + timedelta(minutes=30)

# Create in Work
work_resp = requests.post(f"{BASE}/add", json={
    "title": "Test: Work event",
    "start_iso": tomorrow_start.isoformat(),
    "end_iso": tomorrow_end.isoformat(),
    "notes": "created from notebook",
    "calendar_title": "Work"
}, timeout=10).json()

# Create in Home
home_resp = requests.post(f"{BASE}/add", json={
    "title": "Test: Home event",
    "start_iso": dayafter_start.isoformat(),
    "end_iso": dayafter_end.isoformat(),
    "notes": "created from notebook",
    "calendar_title": "Home"
}, timeout=10).json()

print("WORK  ->", work_resp)
print("HOME  ->", home_resp)

WORK_EVENT_ID = work_resp.get("id")
HOME_EVENT_ID = home_resp.get("id")


WORK  -> {'title': 'Test: Work event', 'start_iso': '2025-10-03T10:00:00-04:00', 'end_iso': '2025-10-03T10:30:00-04:00', 'id': 'F84E2A19-6355-4B39-BD2B-AE3A66B37565', 'calendar': 'Work'}
HOME  -> {'title': 'Test: Home event', 'start_iso': '2025-10-04T10:00:00-04:00', 'end_iso': '2025-10-04T10:30:00-04:00', 'id': '861C1C3B-C6B6-4017-9513-AB9528052CFA', 'calendar': 'Work'}


In [10]:
import requests
from datetime import datetime

BASE = "http://127.0.0.1:8765"

events_30 = requests.get(f"{BASE}/events", params={"days": 30}, timeout=15).json()

def is_holiday_event(e):
    calname = (e.get("calendar") or "").lower()
    return "holiday" in calname  # excludes "US Holidays", "Holidays in India", etc.

filtered = [e for e in events_30 if not is_holiday_event(e)]

print(f"Found {len(filtered)} non-holiday events in the next 30 days\n")
print(f"{'Start':<20} {'End':<20} {'Calendar':<14} Title")
print("-"*90)
for e in sorted(filtered, key=lambda x: x["start_iso"]):
    s = datetime.fromisoformat(e["start_iso"]).strftime("%Y-%m-%d %H:%M")
    en = datetime.fromisoformat(e["end_iso"]).strftime("%Y-%m-%d %H:%M")
    print(f"{s:<20} {en:<20} {str(e.get('calendar') or ''):<14} {e['title']}")


Found 11 non-holiday events in the next 30 days

Start                End                  Calendar       Title
------------------------------------------------------------------------------------------
2025-10-02 11:46     2025-10-02 12:16     Work           Notebook Tagged Event [#work #deepwork #priority-high]
2025-10-03 10:00     2025-10-03 10:30     Work           Test: Work event
2025-10-04 10:00     2025-10-04 10:30     Work           Test: Home event
2025-10-06 00:00     2025-10-06 01:00     Home           TA Office Hours
2025-10-07 16:10     2025-10-07 18:00     Home           TA: Practicum
2025-10-13 00:00     2025-10-13 01:00     Home           TA Office Hours
2025-10-14 16:10     2025-10-14 18:00     Home           TA: Practicum
2025-10-20 00:00     2025-10-20 01:00     Home           TA Office Hours
2025-10-21 16:10     2025-10-21 18:00     Home           TA: Practicum
2025-10-27 00:00     2025-10-27 01:00     Home           TA Office Hours
2025-10-28 16:10     2025-10-28 

separate calendars

In [15]:
import requests
BASE = "http://127.0.0.1:8765"

cals = requests.get(f"{BASE}/calendars", timeout=10).json()
print(f"Found {len(cals)} calendars\n")
for c in cals:
    print(f"{c['title']:<20} writable={c['allows_modifications']} color={c['color_hex']}  id={c['id']}")


Found 4 calendars

Home                 writable=True color=#2c99d3  id=E198547B-614F-4109-B61A-96C56D4DED8A
Birthdays            writable=False color=#6f829f  id=3CAFCE49-B7E6-4643-957A-5D2C095ABDCC
US Holidays          writable=False color=#bf57da  id=BB8C7E8D-EA0D-43BC-B670-5940F97624F2
Work                 writable=True color=#bf57da  id=20E9D11E-48F8-46E7-9CDB-73B49C70FD41


In [16]:
import requests
from datetime import datetime, timedelta

BASE = "http://127.0.0.1:8765"

# Paste the two IDs you saw above:
WORK_ID = "<<<PASTE_WORK_CALENDAR_ID_HERE>>>"
HOME_ID = "<<<PASTE_HOME_CALENDAR_ID_HERE>>>"

def day_at(hour, minute, days_from_now):
    dt = datetime.now().astimezone().replace(hour=hour, minute=minute, second=0, microsecond=0)
    return (dt + timedelta(days=days_from_now))

# Tomorrow 10:00 for Work; Day-after 10:00 for Home
tomorrow_start = day_at(10, 0, 1);  tomorrow_end = tomorrow_start + timedelta(minutes=30)
dayafter_start = day_at(10, 0, 2);  dayafter_end = dayafter_start + timedelta(minutes=30)

work_resp = requests.post(f"{BASE}/add", json={
    "title": "Test: Work event",
    "start_iso": tomorrow_start.isoformat(),
    "end_iso": tomorrow_end.isoformat(),
    "notes": "made with calendar_id",
    "calendar_id": WORK_ID
}, timeout=10).json()

home_resp = requests.post(f"{BASE}/add", json={
    "title": "Test: Home event",
    "start_iso": dayafter_start.isoformat(),
    "end_iso": dayafter_end.isoformat(),
    "notes": "made with calendar_id",
    "calendar_id": HOME_ID
}, timeout=10).json()

print("WORK  ->", work_resp)
print("HOME  ->", home_resp)


WORK  -> {'title': 'Test: Work event', 'start_iso': '2025-10-03T10:00:00-04:00', 'end_iso': '2025-10-03T10:30:00-04:00', 'id': '13FEBFB4-68AA-4578-AED9-727BDAB383D9', 'calendar': 'Work'}
HOME  -> {'title': 'Test: Home event', 'start_iso': '2025-10-04T10:00:00-04:00', 'end_iso': '2025-10-04T10:30:00-04:00', 'id': '42818E11-92F4-4085-83EE-D0B626DF0DC0', 'calendar': 'Work'}


In [17]:
import requests, collections
from datetime import datetime

BASE = "http://127.0.0.1:8765"

events = requests.get(f"{BASE}/events", params={"days": 7}, timeout=10).json()
by_cal = collections.defaultdict(list)
for e in events:
    by_cal[e.get("calendar","(unknown)")].append(e)

for cal_name in sorted(by_cal.keys()):
    print(f"\n=== {cal_name} ===")
    for e in sorted(by_cal[cal_name], key=lambda x: x["start_iso"]):
        s = datetime.fromisoformat(e["start_iso"]).strftime("%Y-%m-%d %H:%M")
        en = datetime.fromisoformat(e["end_iso"]).strftime("%H:%M")
        print(f"- {s} → {en} | {e['title']} (id: {e['id'][:24]})")



=== Home ===
- 2025-10-06 00:00 → 01:00 | TA Office Hours (id: AC7A3416-9489-4CC0-B959-)
- 2025-10-07 16:10 → 18:00 | TA: Practicum (id: 78321BA2-61E0-4B90-9BBC-)

=== US Holidays ===
- 2025-10-02 00:00 → 00:00 | Yom Kippur (id: 8F09EAB1-3910-4D16-991A-)

=== Work ===
- 2025-10-03 10:00 → 10:30 | Test: Work event (id: 13FEBFB4-68AA-4578-AED9-)
- 2025-10-04 10:00 → 10:30 | Test: Home event (id: 42818E11-92F4-4085-83EE-)


In [18]:
import requests
from datetime import datetime

BASE = "http://127.0.0.1:8765"

events_30 = requests.get(f"{BASE}/events", params={"days": 30}, timeout=15).json()

def is_holiday_event(e):
    calname = (e.get("calendar") or "").lower()
    return "holiday" in calname

filtered = [e for e in events_30 if not is_holiday_event(e)]

print(f"Found {len(filtered)} non-holiday events in the next 30 days\n")
print(f"{'Start':<20} {'End':<20} {'Calendar':<14} Title")
print("-"*90)
for e in sorted(filtered, key=lambda x: x["start_iso"]):
    s = datetime.fromisoformat(e["start_iso"]).strftime("%Y-%m-%d %H:%M")
    en = datetime.fromisoformat(e["end_iso"]).strftime("%Y-%m-%d %H:%M")
    print(f"{s:<20} {en:<20} {str(e.get('calendar') or ''):<14} {e['title']}")


Found 10 non-holiday events in the next 30 days

Start                End                  Calendar       Title
------------------------------------------------------------------------------------------
2025-10-03 10:00     2025-10-03 10:30     Work           Test: Work event
2025-10-04 10:00     2025-10-04 10:30     Work           Test: Home event
2025-10-06 00:00     2025-10-06 01:00     Home           TA Office Hours
2025-10-07 16:10     2025-10-07 18:00     Home           TA: Practicum
2025-10-13 00:00     2025-10-13 01:00     Home           TA Office Hours
2025-10-14 16:10     2025-10-14 18:00     Home           TA: Practicum
2025-10-20 00:00     2025-10-20 01:00     Home           TA Office Hours
2025-10-21 16:10     2025-10-21 18:00     Home           TA: Practicum
2025-10-27 00:00     2025-10-27 01:00     Home           TA Office Hours
2025-10-28 16:10     2025-10-28 18:00     Home           TA: Practicum


In [19]:
import requests
BASE = "http://127.0.0.1:8765"

cals = requests.get(f"{BASE}/calendars", timeout=10).json()
print(f"Found {len(cals)} calendars\n")
for c in cals:
    print(f"{c['title']:<20} writable={c['allows_modifications']}  id={c['id']}")


Found 4 calendars

Home                 writable=True  id=E198547B-614F-4109-B61A-96C56D4DED8A
Birthdays            writable=False  id=3CAFCE49-B7E6-4643-957A-5D2C095ABDCC
US Holidays          writable=False  id=BB8C7E8D-EA0D-43BC-B670-5940F97624F2
Work                 writable=True  id=20E9D11E-48F8-46E7-9CDB-73B49C70FD41


calendar test again

In [21]:
import requests
BASE = "http://127.0.0.1:8765"

cals = requests.get(f"{BASE}/calendars", timeout=10).json()
for c in cals:
    print(f"{c['title']:<20} writable={c['allows_modifications']}  id={c['id']}")


Home                 writable=True  id=E198547B-614F-4109-B61A-96C56D4DED8A
Birthdays            writable=False  id=3CAFCE49-B7E6-4643-957A-5D2C095ABDCC
US Holidays          writable=False  id=BB8C7E8D-EA0D-43BC-B670-5940F97624F2
Work                 writable=True  id=20E9D11E-48F8-46E7-9CDB-73B49C70FD41


In [22]:
import requests
from datetime import datetime, timedelta

BASE = "http://127.0.0.1:8765"

WORK_ID = "20E9D11E-48F8-46E7-9CDB-73B49C70FD41"
HOME_ID = "E198547B-614F-4109-B61A-96C56D4DED8A"

def day_at(hour, minute, days_from_now):
    dt = datetime.now().astimezone().replace(hour=hour, minute=minute, second=0, microsecond=0)
    return dt + timedelta(days=days_from_now)

tomorrow_start = day_at(10, 0, 1);  tomorrow_end = tomorrow_start + timedelta(minutes=30)
dayafter_start = day_at(10, 0, 2);  dayafter_end = dayafter_start + timedelta(minutes=30)

work_resp = requests.post(f"{BASE}/add", json={
    "title": "Test: Work event (ID)",
    "start_iso": tomorrow_start.isoformat(),
    "end_iso": tomorrow_end.isoformat(),
    "calendar_id": WORK_ID
}, timeout=10)

home_resp = requests.post(f"{BASE}/add", json={
    "title": "Test: Home event (ID)",
    "start_iso": dayafter_start.isoformat(),
    "end_iso": dayafter_end.isoformat(),
    "calendar_id": HOME_ID
}, timeout=10)

print("WORK:", work_resp.status_code, work_resp.json())
print("HOME:", home_resp.status_code, home_resp.json())


WORK: 200 {'title': 'Test: Work event (ID)', 'start_iso': '2025-10-03T10:00:00-04:00', 'end_iso': '2025-10-03T10:30:00-04:00', 'id': '8788CAC6-334F-434F-AF2A-B634C8D26709', 'calendar': 'Work'}
HOME: 200 {'title': 'Test: Home event (ID)', 'start_iso': '2025-10-04T10:00:00-04:00', 'end_iso': '2025-10-04T10:30:00-04:00', 'id': '9D5A85B6-BF3E-425E-B52D-7A0B6E460893', 'calendar': 'Home'}


In [23]:
import requests, collections
from datetime import datetime

BASE = "http://127.0.0.1:8765"

events = requests.get(f"{BASE}/events", params={"days": 7}, timeout=10).json()
by_cal = collections.defaultdict(list)
for e in events:
    by_cal[e.get("calendar","(unknown)")].append(e)

for cal_name in sorted(by_cal.keys()):
    print(f"\n=== {cal_name} ===")
    for e in sorted(by_cal[cal_name], key=lambda x: x["start_iso"]):
        s = datetime.fromisoformat(e["start_iso"]).strftime("%Y-%m-%d %H:%M")
        en = datetime.fromisoformat(e["end_iso"]).strftime("%H:%M")
        print(f"- {s} → {en} | {e['title']} (id: {e['id'][:24]})")



=== Home ===
- 2025-10-04 10:00 → 10:30 | Test: Home event (ID) (id: 9D5A85B6-BF3E-425E-B52D-)
- 2025-10-06 00:00 → 01:00 | TA Office Hours (id: AC7A3416-9489-4CC0-B959-)
- 2025-10-07 16:10 → 18:00 | TA: Practicum (id: 78321BA2-61E0-4B90-9BBC-)

=== US Holidays ===
- 2025-10-02 00:00 → 00:00 | Yom Kippur (id: 8F09EAB1-3910-4D16-991A-)

=== Work ===
- 2025-10-03 10:00 → 10:30 | Test: Work event (ID) (id: 8788CAC6-334F-434F-AF2A-)
